## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import pandas as pd
df = pd.read_csv("cookie_cats.csv")

In [2]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

p1 = 0.19  
p2 = 0.20
alpha = 0.05
power = 0.8

effect_size = proportion_effectsize(p1, p2)
analysis = NormalIndPower()
n_per_group = analysis.solve_power(effect_size=effect_size,power=power,alpha=alpha, ratio=1)

print(f"Users per group: {int(n_per_group):}")
print(f"Total users: {int(2 * n_per_group):}")

Users per group: 24637
Total users: 49275


For an increase in retention from 19% to 20% with 80% probability, we have to collect data from approximately 49275 players in total.

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
retention_7_version = (df.groupby("version")["retention_7"].mean())
print(retention_7_version)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


Hypothesis:

Ho = no difference in retention on the 7th day between versions: Ho:p30 = p40

Ha = version gate_30 has a better retention on the 7th day than gate_40: Ha: p30>p40

As per results, version gate_30 has a better retention than gate_40.

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


Ho:p_control = p_treatment
Ha: p_control!= p_treatment

Using two-sided test as not specifying which one is expected to be better

In [4]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control = df[df["version"] == "gate_30"]["retention_7"]
treatment = df[df["version"] == "gate_40"]["retention_7"]

successes = np.array([control.sum(), treatment.sum()])
nobs = np.array([len(control), len(treatment)])

z_stat, p_value = proportions_ztest(successes, nobs, alternative = "two-sided")

ci_control = proportion_confint(count=successes[0], nobs=nobs[0], alpha=0.05, method="normal")
ci_treatment = proportion_confint(count=successes[1], nobs=nobs[1], alpha=0.05, method="normal")

print(f"z statistic = {z_stat:.4f}")
print(f"p-value = {p_value:.6f}")
print(f"Confidence interval 95% for control group: [{ci_control[0]:.4f}, {ci_control[1]:.4f}]")
print(f"Confidence interval 95% for treatment group: [{ci_treatment[0]:.4f}, {ci_treatment[1]:.4f}]")

z statistic = 3.1644
p-value = 0.001554
Confidence interval 95% for control group: [0.1866, 0.1938]
Confidence interval 95% for treatment group: [0.1785, 0.1855]


Based on the results obtained, we reject Ho as p-value = 0.001554 <0.05 meaning that there is a statistically significant difference in 7-day retention between two versions. The intervals do not overlap meaning that the treatment group has a lower retention than the control one.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


Ho = 7-day retention does not depend on the game version (game version and retention_7 are independent).

Ha = 7-day retention depends on the game version (there is association between game version and retention_7).

In [5]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df['version'], df['retention_7'])
print(contingency_table)

retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279


In [6]:
contingency_table = np.array([[36198, 8502],
                              [37210, 8279]])

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi2 statistic = {chi2_stat:.4f}")
print(f"p-value = {p_value:.6f}")

Chi2 statistic = 9.9591
p-value = 0.001601


Ho is rejected as p-value <0.05 meaning that there is a statistically significant association between the game versions and 7-day player retention. Basically game version affects the behavoiur of the player. 